In [84]:
import pandas as pd
import datetime as dt

accept = pd.read_csv('../rawData/accepted_2007_to_2018Q4.csv')

In [85]:
# Use columns from Guetta paper, plus fields mentioned by Aiko
cols = ['id', 'loan_amnt', 'funded_amnt', 'term',
        'int_rate', 'grade', 'sub_grade', 'emp_length', 
        'home_ownership', 'annual_inc', 'verification_status', 
        'issue_d', 'loan_status', 'purpose', 'dti', 'delinq_2yrs', 
        'earliest_cr_line', 'open_acc', 'pub_rec', 'fico_range_high', 
        'fico_range_low', 'revol_bal', 'revol_util', 'total_pymnt', 
        'recoveries', 'last_pymnt_d', 'loan_amnt']
accept = accept[cols]

In [86]:
# Remove subtotal rows
def find_weird(x):
    try:
        _ = int(x)
        return(True)
    except:
        return(False)

accept = accept[accept['id'].apply(find_weird)]
accept['id'] = accept['id'].apply(int)

In [87]:
# Turn dates into datetime
def make_dateval(s):
    s = s.split('-')
    return(dt.datetime.strptime(s[1] + s[0] + '01', '%Y%b%d'))

accept['issue_d'] = accept['issue_d'].apply(make_dateval)
accept.loc[accept['last_pymnt_d'].notna(), 'last_pymnt_d'] = \
    accept.loc[accept['last_pymnt_d'].notna(), 'last_pymnt_d'].apply(make_dateval)

In [88]:
accept = accept[accept['issue_d'] >= pd.Timestamp('20090101')]

In [89]:
# Remove loans that have not terminated yet
accept.loc[accept['loan_status']=='Does not meet the credit policy. Status:Fully Paid', 'loan_status'] = 'Fully Paid'
accept.loc[accept['loan_status']=='Does not meet the credit policy. Status:Charged Off', 'loan_status'] = 'Charged Off'
accept = accept[(accept['loan_status'] == 'Fully Paid') | (accept['loan_status'] == 'Charged Off')]

In [90]:
# Split into train and test
accept = accept.sort_values('issue_d')
N = accept.shape[0]
train = accept.iloc[:int(N * .8), :]
test = accept.iloc[int(N * .8):, :]

In [91]:
train.to_csv('../derivedData/train.csv')

In [92]:
test.to_csv('../derivedData/test.csv')